# Conversational RAG

## Intro
* In many Q&A applications we want to allow user to have back-and-forth conversation, meaning that application needs some sort of "memory" of past questions and answers

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Connect with LLM

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [6]:
#!pip install langchain-community langchainhub langchain-chroma bs4

## Basic RAG without memory

In [7]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("./data/be-good.txt")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [8]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [9]:
output = rag_chain.invoke({"input": "What is this article about?"})

In [10]:
output["answer"]

'The article discusses the concept of making something people want as a motto for startups, advising founders not to worry excessively about the business model initially. It explores the idea that creating something people want and not focusing too much on making money can resemble a charity. Examples like Craigslist running successfully with a charitable approach are also provided.'

In [11]:
output = rag_chain.invoke({"input": "What was my previous question about?"})

In [12]:
output["answer"]

'Your previous question was about the idea of betting against benevolence and how powerful forces like benevolence can lead to successful outcomes in various contexts, such as in the case of Internet startups like Google and Microsoft. The discussion also touched on the success of organizations like Craigslist that operate with a benevolent approach, even though they are not structured as charities.'

## Adding memory

In [13]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [14]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [15]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is this article about?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What was my previous question about?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Your previous question was about the general topic or theme of the article.


In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [17]:
conversational_rag_chain.invoke(
    {"input": "What is this article about?"},
    config={
        "configurable": {"session_id": "001"}
    },  # constructs a key "001" in `store`.
)["answer"]

Parent run 2b2ecc13-4056-44e5-b5ea-9094a966498f not found for run baa1cdbf-9e8f-4dcd-af4e-e015efa8121c. Treating as a root run.


'The article discusses the motto of Y Combinator, "Make something people want," and how focusing on creating value for users can lead to success. It also explores the idea of running a business like a charity, using examples like Craigslist, and how this approach can still be successful. The author reflects on the surprising connection between creating something people want and operating like a charity in the business world.'

In [18]:
conversational_rag_chain.invoke(
    {"input": "What was my previous question about?"},
    config={"configurable": {"session_id": "001"}},
)["answer"]

Parent run 1b87cc43-334f-462d-b583-d9ec72795006 not found for run 92207bd3-623c-4d1d-b9af-9ce145f833bf. Treating as a root run.


'Your previous question was about the topic or subject matter of the article.'

In [19]:
for message in store["001"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is this article about?

AI: The article discusses the motto of Y Combinator, "Make something people want," and how focusing on creating value for users can lead to success. It also explores the idea of running a business like a charity, using examples like Craigslist, and how this approach can still be successful. The author reflects on the surprising connection between creating something people want and operating like a charity in the business world.

User: What was my previous question about?

AI: Your previous question was about the topic or subject matter of the article.

